# CHATBOT FOR QUESTION ANSWERS WIKIPEDIA

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
a= pd.read_csv("G:/DATA/Python/Chatbot/question_answer_pairs_a.txt",sep="\t",encoding = 'ISO-8859-1')
b=pd.read_csv("G:/DATA/Python/Chatbot/question_answer_pairs_b.txt",sep="\t",encoding = 'ISO-8859-1')
c= pd.read_csv("G:/DATA/Python/Chatbot/question_answer_pairs_c.txt",sep="\t",encoding = 'ISO-8859-1')

In [3]:
c.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,data/set4/a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,data/set4/a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,data/set4/a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,data/set4/a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,data/set4/a10


In [4]:
#Now we have combine the three datasets into one

In [5]:
df=pd.concat([a,b,c])


In [6]:
df.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,data/set3/a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,data/set3/a4


In [7]:
# We only keep Question and answer column in our dataframe

In [8]:
df=df.loc[:,["Question","Answer"]]


In [9]:
df.iloc[1,]

Question    Was Abraham Lincoln the sixteenth President of...
Answer                                                   Yes.
Name: 1, dtype: object

In [10]:
df.shape

(3998, 2)

In [11]:
# Removing the duplicate entries in the data
df=df.drop_duplicates(subset="Question")

In [12]:
#here we can see that there were multiple duplicate values which got removed 

In [13]:
df.shape

(2457, 2)

In [14]:
#checking for na values

In [15]:
df.isna().sum()

Question      1
Answer      272
dtype: int64

In [16]:
# here we drop NA values
df=df.dropna()

In [17]:
import string

In [18]:
# to check for stopwords we 
from nltk.corpus import stopwords


In [19]:
# UDF for cleaning the data
def text_process(mess):
    """
    1. remove the punctuation
    2. remove the stopwords
    3.return the list of clean textwords
    """
    nopunc=[char for char in mess if char not in string.punctuation]
    nopunc="".join(nopunc)
    
    return[word for word in nopunc.split() if word not in stopwords.words("english")]

In [22]:

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [23]:
#creating a termdocument matrix
tfidf_vectorizer = TfidfVectorizer(analyzer=text_process)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Question'])
print(tfidf_matrix.shape)

(2185, 4387)


In [24]:
def ask_question(question):
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)
    max_similarity = np.argmax(similarity, axis=None)

    print('Your question:', question)
    print('Closest question found:', df.iloc[max_similarity]['Question'])
    print('Similarity: {:.2%}'.format(similarity[0,max_similarity]))
    print('Answer:', df.iloc[max_similarity]['Answer'])

In [25]:
# Testing our chatbot
ask_question("Was Abraham Lincoln the sixteenth President of the United States")

Your question: Was Abraham Lincoln the sixteenth President of the United States
Closest question found: Was Abraham Lincoln the sixteenth President of the United States?
Similarity: 100.00%
Answer: yes
